# Retrieve NCBI Taxonomy Info For RefSeq Genomes

In `../refseq_masher/data/RefSeqSketches.msh`, each sketch ID has an NCBI Taxonomy UID as the 3rd element if you were to split the ID on `-` (dashes):

In [2]:
!mash info -t ../refseq_masher/data/RefSeqSketches.msh | head 

#Hashes	Length	ID	Comment
400	2679921514	./rcn/refseq-AC-10090-PRJNA16113-SAMN03004379-GCF_000002165.2-.-Mus_musculus.fna	
400	16300	./rcn/refseq-AC-10116-PRJNA12455-.-.-.-Rattus_norvegicus.fna	
400	2559974438	./rcn/refseq-AC-10116-PRJNA16219-.-GCF_000002265.2-.-Rattus_norvegicus.fna	
400	30536	./rcn/refseq-AC-10512-PRJNA15101-.-.-.-Canine_adenovirus_1.fna	
400	31323	./rcn/refseq-AC-10514-PRJNA15102-.-.-.-Canine_adenovirus_2.fna	
400	35937	./rcn/refseq-AC-10515-PRJNA15106-.-.-.-Human_adenovirus_2.fna	
400	35514	./rcn/refseq-AC-10519-PRJNA15114-.-.-.-Human_adenovirus_7.fna	
400	34794	./rcn/refseq-AC-10522-PRJNA15115-.-.-.-Human_adenovirus_35.fna	
400	36001	./rcn/refseq-AC-10533-PRJNA15113-.-.-.-Human_adenovirus_1.fna	


Output tabular Mash info output to a file

In [3]:
!mash info -t ../refseq_masher/data/RefSeqSketches.msh > RefSeqSketches.msh-info.tab

Read file into Pandas DataFrame

In [5]:
import pandas as pd

In [6]:
df_mash_info = pd.read_table('RefSeqSketches.msh-info.tab')

In [7]:
df_mash_info.head()

,#Hashes,Length,ID,Comment
0,400,2679921514,./rcn/refseq-AC-10090-PRJNA16113-SAMN03004379-...,NaN
1,400,16300,./rcn/refseq-AC-10116-PRJNA12455-.-.-.-Rattus_...,NaN
2,400,2559974438,./rcn/refseq-AC-10116-PRJNA16219-.-GCF_0000022...,NaN
3,400,30536,./rcn/refseq-AC-10512-PRJNA15101-.-.-.-Canine_...,NaN
4,400,31323,./rcn/refseq-AC-10514-PRJNA15102-.-.-.-Canine_...,NaN


Split the Mash info IDs on `-` (dashes)

In [9]:
import re

In [10]:
ids = []
for mash_id in df_mash_info['ID']:
    r = re.search(r'\./rcn/(.+)', mash_id)
    if r:
        sp = r.group(1).split('-')
        ids.append(sp)

Mash info IDs into a Pandas DataFrame

In [13]:
df_split_ids = pd.DataFrame(ids)

In [14]:
df_split_ids.head()

,0,1,2,3,4,5,6,7
0,refseq,AC,10090,PRJNA16113,SAMN03004379,GCF_000002165.2,.,Mus_musculus.fna
1,refseq,AC,10116,PRJNA12455,.,.,.,Rattus_norvegicus.fna
2,refseq,AC,10116,PRJNA16219,.,GCF_000002265.2,.,Rattus_norvegicus.fna
3,refseq,AC,10512,PRJNA15101,.,.,.,Canine_adenovirus_1.fna
4,refseq,AC,10514,PRJNA15102,.,.,.,Canine_adenovirus_2.fna


Assign more meaningful column names

In [16]:
columns = """
0
prefix
taxid
bioproject
biosample
assembly_accession
plasmid
fna_filename
""".strip().split('\n')

In [17]:
df_split_ids.columns = columns

In [18]:
df_split_ids.head()

,0,prefix,taxid,bioproject,biosample,assembly_accession,plasmid,fna_filename
0,refseq,AC,10090,PRJNA16113,SAMN03004379,GCF_000002165.2,.,Mus_musculus.fna
1,refseq,AC,10116,PRJNA12455,.,.,.,Rattus_norvegicus.fna
2,refseq,AC,10116,PRJNA16219,.,GCF_000002265.2,.,Rattus_norvegicus.fna
3,refseq,AC,10512,PRJNA15101,.,.,.,Canine_adenovirus_1.fna
4,refseq,AC,10514,PRJNA15102,.,.,.,Canine_adenovirus_2.fna


The number of unique NCBI Taxonomy UIDs

In [20]:
df_split_ids.taxid.unique().size

42367

## Fetch NCBI Taxonomy info using BioPython Entrez efetch

Fetch the Taxonomy info from NCBI for a list of NCBI Taxonomy UIDs. 

In [21]:
from Bio import Entrez

Entrez.email = 'peter.kruczkiewicz@gmail.com'

def get_tax_lineage(taxids, filter_no_rank=True):
    assert isinstance(taxids, list)
    h = Entrez.efetch(db='Taxonomy', id=taxids, retmode='xml')
    recs = Entrez.read(h)
    dfs = []
    for taxid, rec in zip(taxids, recs):
        lineage_info = rec['LineageEx']
        lineage_info.append({'TaxId': rec['TaxId'], 
                             'ScientificName': rec['ScientificName'],
                             'Rank': rec['Rank']})
        dflineage = pd.DataFrame(lineage_info)
        if filter_no_rank:
            dflineage = dflineage[dflineage['Rank'] != 'no rank']
        dflineage = dflineage[~dflineage.duplicated(keep='first')]
        dflineage['query_taxid'] = taxid
        dfs.append(dflineage)
    return pd.concat(dfs), recs

Here's what the output looks like for a single taxid, 562, corresponding to *E. coli*

In [23]:
dftest, recs_test = get_tax_lineage(['562'], filter_no_rank=False)

In [27]:
dftest

,Rank,ScientificName,TaxId,query_taxid
0,no rank,cellular organisms,131567,562
1,superkingdom,Bacteria,2,562
2,phylum,Proteobacteria,1224,562
3,class,Gammaproteobacteria,1236,562
4,order,Enterobacterales,91347,562
5,family,Enterobacteriaceae,543,562
6,genus,Escherichia,561,562
7,species,Escherichia coli,562,562


In [25]:
len(recs_test)

1

In [26]:
recs_test

[{'TaxId': '562', 'ScientificName': 'Escherichia coli', 'OtherNames': {'EquivalentName': ['Escherichia/Shigella coli'], 'Synonym': ['Bacillus coli', 'Bacterium coli', 'Bacterium coli commune', 'Enterococcus coli'], 'Acronym': [], 'Misspelling': [], 'Anamorph': [], 'Includes': ['Escherichia sp. MAR', 'bacterium 10a', 'bacterium E3'], 'CommonName': ['E. coli'], 'Inpart': [], 'Misnomer': [], 'Teleomorph': [], 'GenbankSynonym': [], 'GenbankAnamorph': [], 'Name': [{'ClassCDE': 'authority', 'DispName': '"Bacillus coli" Migula 1895'}, {'ClassCDE': 'authority', 'DispName': '"Bacterium coli commune" Escherich 1885'}, {'ClassCDE': 'authority', 'DispName': '"Bacterium coli" (Migula 1895) Lehmann and Neumann 1896'}, {'ClassCDE': 'authority', 'DispName': 'Escherichia coli (Migula 1895) Castellani and Chalmers 1919'}, {'ClassCDE': 'type material', 'DispName': 'ATCC 11775'}, {'ClassCDE': 'type material', 'DispName': 'CCUG 24'}, {'ClassCDE': 'type material', 'DispName': 'CCUG 29300'}, {'ClassCDE': 'ty

### Getting all taxonomic info for all 42367 taxids

The taxonomy info was fetched in batches of 1000 (max taxids you can query at once with the API is 10000).

However there are a few missing taxids that needed to be accounted for:
```
1539978
1608275
1609188
```

Unfortunately, the response from NCBI API doesn't let you know that your taxid was deleted or changed, so it was something that had to be manually tracked down.



*Don't execute the following cell*

In [176]:
# do not execute this cell; it takes a long time
step = 1000
all_dfs = []
all_recs = []
for i in range(0, len(taxids), step):
    retry = 0
    while True:
        try:
            if retry >= 5:
                break
            big_df, recs = get_tax_lineage(taxids[i:i+step], filter_no_rank=False)
            print(i, big_df.query_taxid.unique().size, len(recs))
            all_dfs.append(big_df)
            all_recs += recs
            break
        except Exception as ex:
            print(ex.message)
            retry += 1
            print('Retrying', retry)

0 1000 1000
1000 1000 1000
2000 1000 1000
3000 1000 1000
4000 997 997
5000 1000 1000
6000 1000 1000
7000 1000 1000
8000 1000 1000
9000 1000 1000
10000 1000 1000
11000 1000 1000
12000 1000 1000
13000 1000 1000
14000 1000 1000
15000 1000 1000
16000 1000 1000
17000 1000 1000
18000 1000 1000
19000 1000 1000
20000 1000 1000
21000 1000 1000
22000 1000 1000
23000 1000 1000
24000 1000 1000
25000 1000 1000
26000 1000 1000
27000 1000 1000
28000 1000 1000
29000 1000 1000
30000 1000 1000
31000 1000 1000
32000 1000 1000
33000 1000 1000
34000 1000 1000
35000 1000 1000
36000 1000 1000
37000 1000 1000
38000 1000 1000
39000 1000 1000
40000 1000 1000
41000 1000 1000
42000 367 367


Taxonomy info for 3 of the taxids could not be retrieved

In [33]:
df_split_ids.taxid.unique().size - len(all_recs)

3

The missing taxids were in between taxids 4000 and 5000

In [178]:
missing_recs = all_recs[4000:5000]

Manually determined missing taxids

In [200]:
deleted_taxids = '''
1539978
1608275
1609188
'''.strip().split('\n')

In [201]:
taxids_4000_no_missing = [x for x in taxids[4000:5000] if not x in deleted_taxids]

In [202]:
len(taxids_4000_no_missing)

997

Queried for non-missing taxids in `taxids[4000:5000]`

In [203]:
big_df, recs = get_tax_lineage(taxids_4000_no_missing, filter_no_rank=False)
print(i, big_df.query_taxid.unique().size, len(recs))

4900 997 997


Replaced DataFrame at index 4 with missing taxids accounted for

In [207]:
all_dfs[4] = big_df

### Concatenated DataFrames from batched NCBI Taxonomy DB queries

In [208]:
dfalltaxid = pd.concat(all_dfs)

In [212]:
dfalltaxid = dfalltaxid.reset_index()

In [216]:
dfalltaxid = dfalltaxid[dfalltaxid.columns[1:]]

Removed some uninformative info

In [229]:
dfalltaxid = dfalltaxid[~(dfalltaxid.ScientificName == 'cellular organisms')]

In [233]:
dfalltaxid.Rank[dfalltaxid.Rank == 'no rank'] = None

Rename columns

In [270]:
dfalltaxid.columns = """
rank
name
taxid
query_taxid
""".strip().split('\n')

### Save taxonomy information table

In [234]:
dfalltaxid.to_csv('refseq-taxid-info.csv', index=None)

In [238]:
dfalltaxid[dfalltaxid.query_taxid == '10090']

,Rank,ScientificName,TaxId,query_taxid
1,superkingdom,Eukaryota,2759,10090
2,None,Opisthokonta,33154,10090
3,kingdom,Metazoa,33208,10090
4,None,Eumetazoa,6072,10090
5,None,Bilateria,33213,10090
6,None,Deuterostomia,33511,10090
7,phylum,Chordata,7711,10090
8,subphylum,Craniata,89593,10090
9,None,Vertebrata,7742,10090
10,None,Gnathostomata,7776,10090


## Generate NCBI RefSeq Taxonomy Summary Table

The `dfalltaxid` table contains a lot of extra information in an inefficient format so it would be useful to summarize that info in a table with one row corresponding to one RefSeq taxid.

In [275]:
from typing import List

In [278]:
def get_full_taxonomy(l: List[str]) -> str:
    """From an ordered list of taxonomic classifications, return a string of concatenated taxonomic info

    Remove some of the redundancy between classifications, e.g.

    ["Salmonella enterica", "Salmonella enterica subsp. enterica"]

    is concatenated to

    "Salmonella enterica; subsp. enterica" rather than "Salmonella enterica; Salmonella enterica subsp. enterica"

    Args:
        l: list of taxonomic classifications

    Returns:
        (str): concatenated taxonomic classifications with some redundancy removed
    """
    out = [l[0]]
    for i in range(1, len(l)):
        prev = l[i - 1]
        curr = l[i]
        out.append(curr.replace(prev, '').strip())
    return '; '.join(out)


def summary_taxid(dftax):
    taxids = dftax.query_taxid.unique()
    df_has_rank = dftax[~pd.isnull(dftax['rank'])]
    dicts = []
    for i, taxid in enumerate(taxids):
        d = {'taxid': taxid}
        df_has_rank_taxid = df_has_rank[df_has_rank['query_taxid'] == taxid]
        for _, r in df_has_rank_taxid.iterrows():
            d['taxonomic_{}'.format(r['rank'])] = r['name']
        df_matching_taxid = dftax[dftax['query_taxid'] == taxid]
        # highest resolution taxonomic classification is the last entry for that taxid so get last row
        try:
            _, row = df_matching_taxid[~df_matching_taxid['query_taxid'].duplicated(keep='last')].iterrows().__next__()
            d['top_taxonomy_name'] = row['name']
        except StopIteration:
            d['top_taxonomy_name'] = None
            print('Could not get top taxonomic classification for taxid %s', taxid)
        # build string of concatenated all taxonomic info with some redundancy between successive terms
        try:
            d['full_taxonomy'] = get_full_taxonomy(list(df_matching_taxid['name']))
        except IndexError:
            d['full_taxonomy'] = None
            print('Could not get full taxonomy for taxid %s', taxid)
        dicts.append(d)
        if i % 1000 == 0:
            print('Parsed taxnomic info entries', i, d, len(dicts))
    return pd.DataFrame(dicts)

Produce summarized NCBI Taxonomy info table for each unique taxid in `../refseq_masher/data/RefSeqSketches.msh`

In [279]:
df_summary_taxid = summary_taxid(dfalltaxid)

Parsed taxnomic info entries 0 {'taxid': '10090', 'taxonomic_superkingdom': 'Eukaryota', 'taxonomic_kingdom': 'Metazoa', 'taxonomic_phylum': 'Chordata', 'taxonomic_subphylum': 'Craniata', 'taxonomic_class': 'Mammalia', 'taxonomic_superorder': 'Euarchontoglires', 'taxonomic_order': 'Rodentia', 'taxonomic_suborder': 'Myomorpha', 'taxonomic_family': 'Muridae', 'taxonomic_subfamily': 'Murinae', 'taxonomic_genus': 'Mus', 'taxonomic_subgenus': 'Mus', 'taxonomic_species': 'Mus musculus', 'top_taxonomy_name': 'Mus musculus', 'full_taxonomy': 'Eukaryota; Opisthokonta; Metazoa; Eumetazoa; Bilateria; Deuterostomia; Chordata; Craniata; Vertebrata; Gnathostomata; Teleostomi; Euteleostomi; Sarcopterygii; Dipnotetrapodomorpha; Tetrapoda; Amniota; Mammalia; Theria; Eutheria; Boreoeutheria; Euarchontoglires; Glires; Rodentia; Myomorpha; Muroidea; Muridae; Murinae; Mus; ; musculus'} 1
Parsed taxnomic info entries 1000 {'taxid': '1113547', 'taxonomic_superkingdom': 'Viruses', 'taxonomic_order': 'Caudovir

Parsed taxnomic info entries 14000 {'taxid': '9595', 'taxonomic_superkingdom': 'Eukaryota', 'taxonomic_kingdom': 'Metazoa', 'taxonomic_phylum': 'Chordata', 'taxonomic_subphylum': 'Craniata', 'taxonomic_class': 'Mammalia', 'taxonomic_superorder': 'Euarchontoglires', 'taxonomic_order': 'Primates', 'taxonomic_suborder': 'Haplorrhini', 'taxonomic_infraorder': 'Simiiformes', 'taxonomic_parvorder': 'Catarrhini', 'taxonomic_superfamily': 'Hominoidea', 'taxonomic_family': 'Hominidae', 'taxonomic_subfamily': 'Homininae', 'taxonomic_genus': 'Gorilla', 'taxonomic_species': 'Gorilla gorilla', 'taxonomic_subspecies': 'Gorilla gorilla gorilla', 'top_taxonomy_name': 'Gorilla gorilla gorilla', 'full_taxonomy': 'Eukaryota; Opisthokonta; Metazoa; Eumetazoa; Bilateria; Deuterostomia; Chordata; Craniata; Vertebrata; Gnathostomata; Teleostomi; Euteleostomi; Sarcopterygii; Dipnotetrapodomorpha; Tetrapoda; Amniota; Mammalia; Theria; Eutheria; Boreoeutheria; Euarchontoglires; Primates; Haplorrhini; Simiiforme

Parsed taxnomic info entries 28000 {'taxid': '1339273', 'taxonomic_superkingdom': 'Bacteria', 'taxonomic_phylum': 'Bacteroidetes', 'taxonomic_class': 'Bacteroidia', 'taxonomic_order': 'Bacteroidales', 'taxonomic_family': 'Bacteroidaceae', 'taxonomic_genus': 'Bacteroides', 'taxonomic_species': 'Bacteroides fragilis', 'top_taxonomy_name': 'Bacteroides fragilis str. B1 (UDC16-1)', 'full_taxonomy': 'Bacteria; FCB group; Bacteroidetes/Chlorobi group; Bacteroidetes; Bacteroidia; Bacteroidales; Bacteroidaceae; Bacteroides; fragilis; str. B1 (UDC16-1)'} 28001
Parsed taxnomic info entries 29000 {'taxid': '1388080', 'taxonomic_superkingdom': 'Bacteria', 'taxonomic_phylum': 'Firmicutes', 'taxonomic_class': 'Bacilli', 'taxonomic_order': 'Bacillales', 'taxonomic_family': 'Staphylococcaceae', 'taxonomic_genus': 'Staphylococcus', 'taxonomic_species': 'Staphylococcus aureus', 'top_taxonomy_name': 'Staphylococcus aureus M0131', 'full_taxonomy': 'Bacteria; Terrabacteria group; Firmicutes; Bacilli; Bacil

In [280]:
df_summary_taxid.head()

,full_taxonomy,taxid,taxonomic_class,taxonomic_cohort,taxonomic_family,taxonomic_forma,taxonomic_genus,taxonomic_infraclass,taxonomic_infraorder,taxonomic_kingdom,...,taxonomic_subspecies,taxonomic_subtribe,taxonomic_superclass,taxonomic_superfamily,taxonomic_superkingdom,taxonomic_superorder,taxonomic_superphylum,taxonomic_tribe,taxonomic_varietas,top_taxonomy_name
0,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,10090,Mammalia,NaN,Muridae,NaN,Mus,NaN,NaN,Metazoa,...,NaN,NaN,NaN,NaN,Eukaryota,Euarchontoglires,NaN,NaN,NaN,Mus musculus
1,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,10116,Mammalia,NaN,Muridae,NaN,Rattus,NaN,NaN,Metazoa,...,NaN,NaN,NaN,NaN,Eukaryota,Euarchontoglires,NaN,NaN,NaN,Rattus norvegicus
2,"Viruses; dsDNA viruses, no RNA stage; Adenovir...",10512,NaN,NaN,Adenoviridae,NaN,Mastadenovirus,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Viruses,NaN,NaN,NaN,NaN,Canine adenovirus 1
3,"Viruses; dsDNA viruses, no RNA stage; Adenovir...",10514,NaN,NaN,Adenoviridae,NaN,Mastadenovirus,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Viruses,NaN,NaN,NaN,NaN,Canine adenovirus 2
4,"Viruses; dsDNA viruses, no RNA stage; Adenovir...",10515,NaN,NaN,Adenoviridae,NaN,Mastadenovirus,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Viruses,NaN,NaN,NaN,NaN,Human adenovirus 2


In [281]:
df_summary_taxid.shape

(42364, 32)

### Save NCBI RefSeq Taxonomy Summary Table

In [282]:
df_summary_taxid.to_csv('../refseq_masher/data/ncbi_refseq_taxonomy_summary.csv', index=None)